In [3]:
import requests
import json
r = requests.post(
    'https://api.semanticscholar.org/graph/v1/paper/batch',
    params={'fields': 'referenceCount,citationCount,title'},
    json={"ids": ["649def34f8be52c8b66281af98ae884c09aef38b", "ARXIV:2106.15928"]}
)
print(json.dumps(r.json(), indent=2))

[
  {
    "paperId": "649def34f8be52c8b66281af98ae884c09aef38b",
    "title": "Construction of the Literature Graph in Semantic Scholar",
    "referenceCount": 27,
    "citationCount": 299
  },
  {
    "paperId": "f712fab0d58ae6492e3cdfc1933dae103ec12d5d",
    "title": "Reinfection and low cross-immunity as drivers of epidemic resurgence under high seroprevalence: a model-based approach with application to Amazonas, Brazil",
    "referenceCount": 13,
    "citationCount": 0
  }
]

[
  {
    "paperId": "649def34f8be52c8b66281af98ae884c09aef38b",
    "title": "Construction of the Literature Graph in Semantic Scholar",
    "referenceCount": 27,
    "citationCount": 376
  },
  {
    "paperId": "f712fab0d58ae6492e3cdfc1933dae103ec12d5d",
    "title": "Reinfection and low cross-immunity as drivers of epidemic resurgence under high seroprevalence: a model-based approach with application to Amazonas, Brazil",
    "referenceCount": 13,
    "citationCount": 0
  }
]


[{'paperId': '649def34f8be52c8b66281af98ae884c09aef38b',
  'title': 'Construction of the Literature Graph in Semantic Scholar',
  'referenceCount': 27,
  'citationCount': 299},
 {'paperId': 'f712fab0d58ae6492e3cdfc1933dae103ec12d5d',
  'title': 'Reinfection and low cross-immunity as drivers of epidemic resurgence under high seroprevalence: a model-based approach with application to Amazonas, Brazil',
  'referenceCount': 13,
  'citationCount': 0}]

In [1]:
from openai import OpenAI
import os
client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY')
)
models = client.models.list()
for model in models:
    print(model.id)


gpt-4o-audio-preview-2024-10-01
gpt-4o-mini-audio-preview
o3-mini-2025-01-31
o3-mini
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
gpt-4o-mini-realtime-preview
dall-e-2
gpt-4o-2024-08-06
o1
gpt-3.5-turbo
o1-preview-2024-09-12
gpt-3.5-turbo-0125
o1-preview
text-embedding-ada-002
o1-mini-2024-09-12
gpt-4o
whisper-1
dall-e-3
gpt-3.5-turbo-16k-0613
gpt-4
gpt-4-1106-preview
babbage-002
omni-moderation-latest
omni-moderation-2024-09-26
tts-1-hd-1106
gpt-4o-2024-05-13
tts-1-hd
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
chatgpt-4o-latest
gpt-4-0613
gpt-4o-2024-11-20
gpt-4-turbo
tts-1
tts-1-1106
davinci-002
gpt-4-turbo-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-audio-preview
text-embedding-3-small
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-1106
gpt-4o-realtime-preview-2024-10-01
gpt-3.5-turbo-16k
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4o-audio-preview-2024-12-17
text-embedding-3-large
gpt-4o-realtime-preview-2024-12-17
gpt-4o-realtime-preview
gpt-4-0125-preview
o1-mini


In [7]:
def create_chat_completion(
    messages,
    google=False,
    model="gpt-4o",
    frequency_penalty=0,
    logit_bias=None,
    logprobs=False,
    top_logprobs=None,
    max_tokens=None,
    n=1,
    presence_penalty=0,
    response_format=None,
    seed=None,
    stop=None,
    stream=False,
    include_usage=False,
    temperature=1,
    tools=None,
    tool_choice="none",
    user=None,
    reasoning_effort="medium"  # New parameter: accepts "low", "medium", or "high"
):
    if google:
        client = OpenAI(
            api_key=os.getenv("GOOGLE_API_KEY"),
            base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        )
        params = {
            "model": model,
            "messages": messages,
            "stream": stream,
            "temperature": temperature,
        }
    else:
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        params = {
            "model": model,
            "messages": messages,
            "frequency_penalty": frequency_penalty,
            "logit_bias": logit_bias,
            "logprobs": logprobs,
            "top_logprobs": top_logprobs,
            "max_tokens": max_tokens,
            "n": n,
            "presence_penalty": presence_penalty,
            "response_format": response_format,
            "seed": seed,
            "stop": stop,
            "stream": stream,
            "temperature": temperature,
            "user": user,
            "response_format": response_format,
        }



    if model == "o3-mini":
        # Insert the reasoning effort into the request parameters.
        params["reasoning_effort"] = reasoning_effort
        params.pop("temperature", None)
    if stream:
        params["stream_options"] = {"include_usage": include_usage}
    else:
        params.pop("stream_options", None)
    if tools:
        params["tools"] = [{"type": "function", "function": tool} for tool in tools]
        params["tool_choice"] = tool_choice
    if response_format == "json_object":
        params["response_format"] = {"type": "json_object"}
    elif response_format == "text":
        params["response_format"] = {"type": "text"}
    params = {k: v for k, v in params.items() if v is not None}

    completion = client.chat.completions.create(**params)

    # If not streaming and usage details are available, extract reasoning tokens.
    if not stream and "usage" in completion:
        usage = completion.get("usage", {})
        completion_tokens_details = usage.get("completion_tokens_details", {})
        completion["reasoning_tokens"] = completion_tokens_details.get("reasoning_tokens")

    return completion


In [8]:
response = create_chat_completion(model="o3-mini", messages=[{"role": "system", "content": "What is the capital of France?"}], reasoning_effort="low")
print(response.choices[0].message.content)

TypeError: Completions.create() got an unexpected keyword argument 'reasoning_effort'

In [11]:
from openai import OpenAI
import os
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

response = client.chat.completions.create(
  model="o3-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What are the GDMT meds for CHF?"
        }
      ]
    },
    
  ],
  response_format={
    "type": "text"
  },
  reasoning_effort="low",
  max_completion_tokens = 4000
)

In [12]:
print(response.choices[0].message.content)

Guideline‐directed medical therapy (GDMT) for heart failure with reduced ejection fraction (HFrEF) is based on large clinical trials that have shown improved survival, reduced hospitalizations, and better quality of life. The key medication classes include:

1. ACE Inhibitors (or ARBs/ARNIs)  
 • ACE Inhibitors (e.g., lisinopril, enalapril) help reduce afterload and remodeling.  
 • Angiotensin Receptor Blockers (ARBs, e.g., losartan, valsartan) are alternatives if ACE inhibitors aren’t tolerated.  
 • Angiotensin Receptor Neprilysin Inhibitors (ARNIs, e.g., sacubitril/valsartan) have shown additional benefits compared with ACE inhibitors and are recommended for eligible patients.

2. Beta Blockers  
 • Beta blockers (e.g., carvedilol, metoprolol succinate, bisoprolol) reduce sympathetic activation, limit disease progression, and improve survival.

3. Mineralocorticoid Receptor Antagonists (MRAs)  
 • MRAs (e.g., spironolactone, eplerenone) help counteract aldosterone’s effects, decrea

In [13]:
print(response.usage.completion_tokens_details.reasoning_tokens)

128


In [ ]:
import requests
import json
r = requests.post(
    'https://api.semanticscholar.org/graph/v1/paper/batch',
    params={'fields': 'referenceCount,citationCount,title'},
    json={"ids": ["649def34f8be52c8b66281af98ae884c09aef38b", "ARXIV:2106.15928"]}
)
print(json.dumps(r.json(), indent=2))

[
  {
    "paperId": "649def34f8be52c8b66281af98ae884c09aef38b",
    "title": "Construction of the Literature Graph in Semantic Scholar",
    "referenceCount": 27,
    "citationCount": 299
  },
  {
    "paperId": "f712fab0d58ae6492e3cdfc1933dae103ec12d5d",
    "title": "Reinfection and low cross-immunity as drivers of epidemic resurgence under high seroprevalence: a model-based approach with application to Amazonas, Brazil",
    "referenceCount": 13,
    "citationCount": 0
  }
]

In [28]:
# Input parameters for the Semantic Scholar API query
api_base_url = "https://api.semanticscholar.org/graph/v1/paper/search"
query = "covid vaccination"  # Default search query
fields = "title,authors,citations.abstract"  # Default fields to return
offset = 0  # Default offset for pagination
limit = 3  # Default limit of results to return (must be <= 100)
publication_types = "JournalArticle"  # Optional: e.g., "JournalArticle,Review"
open_access_pdf = False  # Set to True to restrict to papers with open access PDFs
min_citation_count = None  # Optional: minimum number of citations
publication_date_or_year = None  # Optional: e.g., "2016-03-05:2020-06-06"
year = "2023"  # Optional: e.g., "2016-2020"
venue = None  # Optional: e.g., "Nature,Radiology"
fields_of_study = None  # Optional: e.g., "Physics,Mathematics"

# Construct the query parameters
params = {
    "query": query,
    "fields": fields,
    "offset": offset,
    "limit": limit
}

# Add optional parameters if specified
if publication_types:
    params["publicationTypes"] = publication_types
if open_access_pdf:
    params["openAccessPdf"] = ""
if min_citation_count is not None:
    params["minCitationCount"] = min_citation_count
if publication_date_or_year:
    params["publicationDateOrYear"] = publication_date_or_year
if year:
    params["year"] = year
if venue:
    params["venue"] = venue
if fields_of_study:
    params["fieldsOfStudy"] = fields_of_study

In [38]:
import requests

# Execute the API query
response = requests.get(api_base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    total_papers = data.get("total", 0)
    papers = data.get("data", [])
    
    # Display the total number of papers found
    print(f"Total papers found: {total_papers}")
    
    # Display information about each paper
    for paper in papers:
        paper_id = paper.get("paperId", "N/A")
        title = paper.get("title", "N/A")
        authors = ", ".join([author.get("name", "N/A") for author in paper.get("authors", [])])
        print(f"\nPaper ID: {paper_id}\nTitle: {title}\nAuthors: {authors}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Total papers found: 154992

Paper ID: df72552a9579aac7e1b3f9ef25d6f1b9ee675d20
Title: Long COVID risk and pre-COVID vaccination in an EHR-based cohort study from the RECOVER program
Authors: M. Brannock, Robert F. Chew, Alexander J. Preiss, Emily C Hadley, Signe Redfield, J. McMurry, Peter J Leese, A. Girvin, M. Crosskey, A. Zhou, R. Moffitt, M. J. Funk, E. Pfaff, M. Haendel, C. Chute

Paper ID: 3a309f89ad7f1a9b3fa8de16e7231a15487756c8
Title: Connectedness of COVID vaccination with economic policy uncertainty, oil, bonds, and sectoral equity markets: evidence from the US
Authors: Imran Yousaf, Saba Qureshi, Fiza Qureshi, Mariya Gubareva

Paper ID: 115238ce0d461c22fa4fe0556c21d02263ffc11b
Title: The impact of COVID-19 and COVID vaccination on cardiovascular outcomes
Authors: Zubair Akhtar, Mallory J. Trent, A. Moa, T. Tan, Ole Fröbert, C. Macintyre


In [47]:
# Input parameters for the Semantic Scholar API query
api_base_url = "https://api.semanticscholar.org/graph/v1/paper/search"
query = "covid vaccination"  # Default search query
fields = "title,authors"  # Default fields to return
offset = 0  # Default offset for pagination
limit = 3  # Default limit of results to return (must be <= 100)
publication_types = ""  # Optional: e.g., "JournalArticle,Review"
open_access_pdf = False  # Set to True to restrict to papers with open access PDFs
min_citation_count = None  # Optional: minimum number of citations
publication_date_or_year = None  # Optional: e.g., "2016-03-05:2020-06-06"
year = None  # Optional: e.g., "2016-2020"
venue = None  # Optional: e.g., "Nature,Radiology"
fields_of_study = None  # Optional: e.g., "Physics,Mathematics"

# Construct the query parameters
params = {
    "query": query,
    "fields": fields,
    "offset": offset,
    "limit": limit
}

# Add optional parameters if specified
if publication_types:
    params["publicationTypes"] = publication_types
if open_access_pdf:
    params["openAccessPdf"] = ""
if min_citation_count is not None:
    params["minCitationCount"] = min_citation_count
if publication_date_or_year:
    params["publicationDateOrYear"] = publication_date_or_year
if year:
    params["year"] = year
if venue:
    params["venue"] = venue
if fields_of_study:
    params["fieldsOfStudy"] = fields_of_study

In [57]:
import requests
import time

# Execute the initial API query to search for papers
response = requests.get(api_base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    total_papers = data.get("total", 0)
    papers = data.get("data", [])
    
    # Display the total number of papers found
    print(f"Total papers found: {total_papers}")
    
    # Base URL for retrieving paper details
    paper_details_base_url = "https://api.semanticscholar.org/graph/v1/paper/"
    
    # Iterate over each paper to retrieve additional details
    for paper in papers:
        paper_id = paper.get("paperId", "N/A")
        
        # Request additional details for each paper, including URL and external IDs
        paper_details_response = requests.get(
            f"{paper_details_base_url}{paper_id}",
            params={"fields": "title,authors,url,externalIds"}
        )
        
        if paper_details_response.status_code == 200:
            paper_details = paper_details_response.json()
            title = paper_details.get("title", "N/A")
            authors = ", ".join([author.get("name", "N/A") for author in paper_details.get("authors", [])])
            url = paper_details.get("url", "N/A")
            external_ids = paper_details.get("externalIds", {})
            
            # Display each paper's information followed by its URL and external IDs
            print(f"\nPaper ID: {paper_id}\nTitle: {title}\nAuthors: {authors}\nURL: {url}\nExternal IDs: {external_ids}")
        else:
            print(f"Error retrieving details for paper ID {paper_id}: {paper_details_response.status_code}")
        
        # Wait for one second to avoid hitting rate limits
        time.sleep(1)
else:
    print(f"Error: {response.status_code} - {response.text}")

Total papers found: 1361432
Error retrieving details for paper ID df72552a9579aac7e1b3f9ef25d6f1b9ee675d20: 429

Paper ID: 3a309f89ad7f1a9b3fa8de16e7231a15487756c8
Title: Connectedness of COVID vaccination with economic policy uncertainty, oil, bonds, and sectoral equity markets: evidence from the US
Authors: Imran Yousaf, Saba Qureshi, Fiza Qureshi, Mariya Gubareva
URL: https://www.semanticscholar.org/paper/3a309f89ad7f1a9b3fa8de16e7231a15487756c8
External IDs: {'PubMedCentral': '10032274', 'DOI': '10.1007/s10479-023-05267-9', 'CorpusId': 257712018, 'PubMed': '37361093'}

Paper ID: 115238ce0d461c22fa4fe0556c21d02263ffc11b
Title: The impact of COVID-19 and COVID vaccination on cardiovascular outcomes
Authors: Zubair Akhtar, Mallory J. Trent, A. Moa, T. Tan, Ole Fröbert, C. Macintyre
URL: https://www.semanticscholar.org/paper/115238ce0d461c22fa4fe0556c21d02263ffc11b
External IDs: {'PubMedCentral': '10021497', 'DOI': '10.1093/eurheartjsupp/suac123', 'CorpusId': 256918421, 'PubMed': '3693

In [58]:
# Input parameters for the Semantic Scholar API query
api_base_url = "https://api.semanticscholar.org/graph/v1/paper/search"
query = "covid vaccination"  # Default search query
fields = "title,authors"  # Default fields to return
offset = 0  # Default offset for pagination
limit = 3  # Default limit of results to return (must be <= 100)
publication_types = ""  # Optional: e.g., "JournalArticle,Review"
open_access_pdf = False  # Set to True to restrict to papers with open access PDFs
min_citation_count = None  # Optional: minimum number of citations
publication_date_or_year = None  # Optional: e.g., "2016-03-05:2020-06-06"
year = None  # Optional: e.g., "2016-2020"
venue = None  # Optional: e.g., "Nature,Radiology"
fields_of_study = None  # Optional: e.g., "Physics,Mathematics"

# Construct the query parameters
params = {
    "query": query,
    "fields": fields,
    "offset": offset,
    "limit": limit
}

# Add optional parameters if specified
if publication_types:
    params["publicationTypes"] = publication_types
if open_access_pdf:
    params["openAccessPdf"] = ""
if min_citation_count is not None:
    params["minCitationCount"] = min_citation_count
if publication_date_or_year:
    params["publicationDateOrYear"] = publication_date_or_year
if year:
    params["year"] = year
if venue:
    params["venue"] = venue
if fields_of_study:
    params["fieldsOfStudy"] = fields_of_study

In [93]:
import requests
import time

# Execute the initial API query to search for papers
response = requests.get(api_base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    total_papers = data.get("total", 0)
    papers = data.get("data", [])
    
    # Display the total number of papers found
    print(f"Total papers found: {total_papers}")
    
    # Base URL for retrieving paper details
    paper_details_base_url = "https://api.semanticscholar.org/graph/v1/paper/"
    
    # List to store detailed paper information
    detailed_papers_info = []
    
    # Iterate over each paper to retrieve additional details
    for paper in papers:
        paper_id = paper.get("paperId", "N/A")
        
        # Request additional details for each paper, including URL and external IDs
        paper_details_response = requests.get(
            f"{paper_details_base_url}{paper_id}",
            params={"fields": "title,authors,url,externalIds,citations"}
        )
        
        if paper_details_response.status_code == 200:
            paper_details = paper_details_response.json()
            title = paper_details.get("title", "N/A")
            authors = ", ".join([author.get("name", "N/A") for author in paper_details.get("authors", [])])
            url = paper_details.get("url", "N/A")
            external_ids = paper_details.get("externalIds", {})
            citations = paper_details.get("citations", [])
            
            # Store the detailed paper information
            detailed_papers_info.append({
                "paperId": paper_id,
                "title": title,
                "authors": authors,
                "url": url,
                "externalIds": external_ids,
                "citations": citations
            })
            
            # Display each paper's information followed by its URL and external IDs
            print(f"\nPaper ID: {paper_id}\nTitle: {title}\nAuthors: {authors}\nURL: {url}\nExternal IDs: {external_ids}")
        else:
            print(f"Error retrieving details for paper ID {paper_id}: {paper_details_response.status_code}")
        
        # Wait for one second to avoid hitting rate limits
        time.sleep(1)
else:
    print(f"Error: {response.status_code} - {response.text}")

Total papers found: 1361433
Error retrieving details for paper ID df72552a9579aac7e1b3f9ef25d6f1b9ee675d20: 429

Paper ID: 3a309f89ad7f1a9b3fa8de16e7231a15487756c8
Title: Connectedness of COVID vaccination with economic policy uncertainty, oil, bonds, and sectoral equity markets: evidence from the US
Authors: Imran Yousaf, Saba Qureshi, Fiza Qureshi, Mariya Gubareva
URL: https://www.semanticscholar.org/paper/3a309f89ad7f1a9b3fa8de16e7231a15487756c8
External IDs: {'PubMedCentral': '10032274', 'DOI': '10.1007/s10479-023-05267-9', 'CorpusId': 257712018, 'PubMed': '37361093'}
Error retrieving details for paper ID 115238ce0d461c22fa4fe0556c21d02263ffc11b: 429


In [94]:
# Function to generate a URL based on the preferred order of external IDs
def generate_preferred_url(external_ids):
    if 'PubMedCentral' in external_ids:
        return f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{external_ids['PubMedCentral']}/"
    elif 'DOI' in external_ids:
        return f"https://doi.org/{external_ids['DOI']}"
    elif 'CorpusId' in external_ids:
        return f"https://www.semanticscholar.org/paper/{external_ids['CorpusId']}"
    elif 'PubMed' in external_ids:
        return f"https://pubmed.ncbi.nlm.nih.gov/{external_ids['PubMed']}/"
    return None

# Process each paper's citations to generate a list of URLs
citation_urls = []
for paper_info in detailed_papers_info:
    for citation in paper_info.get("citations", []):
        citation_external_ids = citation.get("externalIds", {})
        citation_url = generate_preferred_url(citation_external_ids)
        if citation_url:
            citation_urls.append(citation_url)

# Display the list of URLs
print("\nCitation URLs:")
for url in citation_urls:
    print(url)


Citation URLs:
